# AES-FFT 

## Initialization

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(formatter={'int':hex})
from siha import *

inputFileName = 'data/inputSignal.bin'

# FFT Configuration Data
fft_config = np.array([0x00005F0F, 0x00005F0F, 0x00005F0F, 0x00005F0F], dtype=np.uint32)

# AES Key Configuration Data
encKeybuff = np.array([
    0x0c0d0e0f, 0x08090a0b, 0x04050607, 0x00010203,
    0x00000001, 0x00000000, 0x00000000, 0x00000000], dtype=np.uint32)
decKeybuff = np.array([
    0x0c0d0e0f, 0x08090a0b, 0x04050607, 0x00010203,
    0x00000000, 0x00000000, 0x00000000, 0x00000000], dtype=np.uint32);

BUFFSIZE = 0x400000
inbuff = np.fromfile(inputFileName, dtype=np.uint32)[:BUFFSIZE]
size = inbuff.shape[0]
inbuff1 = inbuff.astype(np.int16);

## AES128 acceleration

In [ ]:
slot_AES128 = 0 # change slot number here if required

offset_key = 0
offset_input = 64
offset_result = 64
size_config = len(decKeybuff)
tid_conf = 1
tid_run = 0

AESaccel = AcapdAccel("AES128", slot_AES128)
AESaccel.loadData(decKeybuff, offset_key)
AESaccel.loadData(inbuff, offset_input)
AESaccel.startAccel()
AESaccel.config(offset_key, size_config, tid_conf)
AESaccel.run(offset_input, size, tid_run, offset_result)
outbuff = AESaccel.retrieve(offset_result, size)
origSignal = np.frombuffer(outbuff.tobytes(), dtype=np.uint32)
AESaccel.close();

# Plot
origSignal0 = origSignal.astype(np.int16)
plt.figure(figsize=(15,5))
plt.suptitle("AES128 Accelerator")
plt.subplot(121)
plt.title('Input to accel Time Series Signal - inputFile')
plt.ylabel('Amplitude')
plt.xlabel('Time')
plt.plot(inbuff1[1000:1500])
plt.subplot(122)
plt.title('Output from accel Time Series Signal - origSignal')
plt.ylabel('Amplitude')
plt.xlabel('Time')
plt.plot(origSignal0[1000:1500])

print("The below plot depicts data sent to AES128 accelerator and recieved from it.");

## FFT acceleration

In [ ]:
slot_FFT = 1 # change slot number here if required

offset_config = 0
size_config = 1
offset_input = 1024
offset_result = 1024
tid_run = 0

FFTaccel = AcapdAccel("FFT", slot_FFT)
FFTaccel.loadData(fft_config, offset_config)
FFTaccel.loadData(origSignal, offset_input)
FFTaccel.config_fft(offset_config, size_config)
FFTaccel.run(offset_input, size, tid_run, offset_result)
FFTofOrigSignal = FFTaccel.retrieve(offset_result, size)
FFTaccel.close();

# Plot
N = 1024*4
x = np.linspace(0, 10**9/2, int(N/2))
f = np.frombuffer(FFTofOrigSignal.tobytes(), dtype=np.int16)
origSignal0FFT = np.sqrt(np.square(f[0::2].astype(np.float64)) + np.square(f[1::2].astype(np.float64)))
i = 0

plt.figure(figsize=(15,5))
plt.suptitle("FFT Accelerator")
plt.subplot(121)
plt.title('Input to accel Time Series Signal - origSignal')
plt.ylabel('Amplitude')
plt.xlabel('Time')
plt.plot(origSignal0[1000:1500])
plt.subplot(122)
plt.title('Output from accel - FFTofOrigSignal')
plt.ylabel('E')
plt.xlabel('Frequency')
plt.plot(x, np.abs(origSignal0FFT[i*N:i*N+int(N/2)]))


print("The below plot depicts data sent to FFT accelerator and recieved from it.");